# **Real time face recognition**

## 1. Install necessary dependencies 

In [ ]:
#!pip install cmake
#!!pip install dlib
#!pip install face_recognition

## 2. Import the libraries

In [1]:
import cv2
import face_recognition
import os
import numpy as np

## 3. Read the images uploaded and store them in an array

In [ ]:
#Set the path to the folder containing the images
folder_path = "./"  
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]


# Initialize arrays to hold the face encodings and their names
known_face_encodings = []
known_face_names = []

# Get a list of all the image files in the folder
image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg')]


## 4. Generate the face encodings with `face_recognition` library

In [2]:
    
# Loop through each image file and generate face encodings
for image_file in image_files:
    # Load the image file
    image_path = os.path.join(folder_path, image_file)
    image = face_recognition.load_image_file(image_path)

    # Generate the face encoding
    face_encodings = face_recognition.face_encodings(image)

    # If there's no face in the image, skip it
    if len(face_encodings) == 0:
        print("No face found in image", image_file)
        continue

    # Get the first face encoding (assuming there's only one face in the image)
    face_encoding = face_encodings[0]

    # Get the name of the person from the image filename (assuming the filename is in the format "name.jpg")
    name = os.path.splitext(image_file)[0]

    # Add the face encoding and name to their respective arrays
    known_face_encodings.append(face_encoding)
    known_face_names.append(name)

## 5. Real time demo

In [ ]:
# Get a reference to webcam #0 (the default one)
video_capture = cv2.VideoCapture(0)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
face_probabilities = []
process_this_frame = True

while True:
    # Grab a single frame of video
    ret, frame = video_capture.read()

    # Only process every other frame of video to save time
    if process_this_frame:
        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        face_probabilities = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            probability = 0.0

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                probability = 1 - face_distances[best_match_index]

            face_names.append(name)
            face_probabilities.append(probability)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name, probability in zip(face_locations, face_names, face_probabilities):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name and probability below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        label = f"{name} ({probability:.2f})"
        cv2.putText(frame, label, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()